In [1]:
import numpy as np
import shutil
import os

import sys
sys.path.append("../utils/")
sys.path.append("../calculating_predictions")
sys.path.append("../evaluation/")
from calculate_predictions import calculate_predictions
from evaluate import evaluate
from run_evaluation import *
from argparse import Namespace
from catboost import CatBoostRegressor, CatBoostClassifier

%reload_ext autoreload
%autoreload 2

In [2]:
def get_baseline_metric(train_days):
    args = Namespace()
    args.verbose = False
    args.first_feature = 0
    args.last_feature = -1
    args.data_folder = "../../../data/best_features_days_data/"
    args.out_folder = "res"
    args.type = "regression"
    args.metric_folder = "metrics"
    args.model_constructor=lambda verbose: CatBoostRegressor(verbose=verbose)
    args.additional_features = None
    args.train_days = train_days
    args.validation_day = None
    args.test_day = 6
    
    def clear(folder):
        if os.path.exists(folder):
            shutil.rmtree(folder)
        os.mkdir(folder)
    
    clear(args.out_folder)
    clear(args.metric_folder)
    
    calculate_predictions(args)
    evaluate(args.out_folder, args.data_folder, args.metric_folder, argmax_positions)
    
    with open("metrics/metrics.txt") as handler:
        metric = float(next(handler).strip().split()[0])
        return metric

In [3]:
train_variants = []
days_num = 7
for train_len in range(1, days_num):
    for train_start in range(days_num - train_len):
        train_variants.append(list(range(train_start, train_start + train_len)))

In [6]:
train_variants

[[0],
 [1],
 [2],
 [3],
 [4],
 [5],
 [0, 1],
 [1, 2],
 [2, 3],
 [3, 4],
 [4, 5],
 [0, 1, 2],
 [1, 2, 3],
 [2, 3, 4],
 [3, 4, 5],
 [0, 1, 2, 3],
 [1, 2, 3, 4],
 [2, 3, 4, 5],
 [0, 1, 2, 3, 4],
 [1, 2, 3, 4, 5],
 [0, 1, 2, 3, 4, 5]]

In [7]:
val_metrics

{'[0]': [0.6486348772082279,
  0.6434834972189899,
  0.616039595380493,
  0.6127978090787002,
  0.6144941303989019,
  0.6078933209524364,
  0.6118782191375862,
  0.6388553222083196,
  0.6341437474245392,
  0.6190676124433443,
  0.6237179745931554,
  0.6168257259918111,
  0.6378954182852462,
  0.6087470443683372,
  0.6289192647571247],
 '[1]': [0.6603045507395411,
  0.6373630757262343,
  0.6622128346705323,
  0.5935024886198038,
  0.6413983236192098,
  0.6316182942481013,
  0.6173931257016531,
  0.6313879720071446,
  0.640106644369489,
  0.6625492827401446,
  0.6315276357408723,
  0.6383243904538912,
  0.6248496877539488,
  0.640985343738549,
  0.6238696050715196],
 '[2]': [0.6277655631963023,
  0.6345333355548995,
  0.6231093252407364,
  0.6401734071094984,
  0.6083912517733522,
  0.6189714581541266,
  0.6276154418345609,
  0.6065575030083311,
  0.6335153846004953,
  0.6370928194777725,
  0.6166602620664234,
  0.6035154719816735,
  0.6026642405288086,
  0.636687754304353,
  0.617039433

In [8]:
no_val_metrics

{'[0]': [0.6310885951371434,
  0.6259196270913501,
  0.600652047682537,
  0.6150582088303054,
  0.6124156910103326,
  0.6138317295702088,
  0.610542228081592,
  0.6165875189429555,
  0.6323319130616465,
  0.6219348366976615,
  0.6148616633326116,
  0.6245396550492498,
  0.6209133974977843,
  0.6431026241284903,
  0.6120363081348161],
 '[1]': [0.6326522725235878,
  0.6568211485072919,
  0.6329688194755883,
  0.6232585355534183,
  0.6496639909130008,
  0.6376908546370841,
  0.6304307400573159,
  0.6239550006697527,
  0.6459937398672365,
  0.6404351527253246,
  0.6047481952884778,
  0.6381937227298337,
  0.6608607367595847,
  0.6146093828632907,
  0.6026051205820178],
 '[2]': [0.6335275356310699,
  0.6373454603812776,
  0.6293376463674002,
  0.627925237474941,
  0.5996004031325913,
  0.6376868724764715,
  0.6269699389360359,
  0.6328221942861468,
  0.6209139647320063,
  0.6319453530990644,
  0.6459766998668285,
  0.587093145583584,
  0.6440565119102972,
  0.6254623137872464,
  0.636277640

In [13]:
for train_variant in train_variants:
    values = val_metrics[str(train_variant)] + no_val_metrics[str(train_variant)]
    print(str(train_variant), np.mean(values), np.std(values))

[0] 0.62197365345653 0.01186870836077553
[1] 0.6344093556117815 0.01707131673771235
[2] 0.6250411190063948 0.014137766806888203
[3] 0.6162317668318499 0.014241861077996024
[4] 0.6302092703656378 0.011294963233404393
[5] 0.6204828594705972 0.015523818698089734
[0, 1] 0.629519819848534 0.015466522411455708
[1, 2] 0.6360107874547777 0.019481518326424176
[2, 3] 0.6178580342723262 0.014995707751954494
[3, 4] 0.6279828571511674 0.016192155590580747
[4, 5] 0.6253024707835829 0.01413793702699328
[0, 1, 2] 0.6242088907693878 0.01743589321960779
[1, 2, 3] 0.6257403334683215 0.014887614354396941
[2, 3, 4] 0.6287080301121535 0.013221164448148142
[3, 4, 5] 0.6204879507700557 0.016986388025609287
[0, 1, 2, 3] 0.6152455657863892 0.020116956061358687
[1, 2, 3, 4] 0.631000140733376 0.014978774846026121
[2, 3, 4, 5] 0.6244940114261675 0.016931154063001667
[0, 1, 2, 3, 4] 0.6274769185733058 0.014958641358263641
[1, 2, 3, 4, 5] 0.6316839843032813 0.014016815409820907
[0, 1, 2, 3, 4, 5] 0.6277195040596317 

In [4]:
%%time
val_metrics = {}
for train_variant in train_variants:
    metric = [get_baseline_metric(train_variant) for i in range(15)]
    val_metrics[str(train_variant)] = metric

"2018-06-18 01:43:58.706490": preprocesing started
"2018-06-18 01:44:08.497058": train features shape: (22801, 80)
"2018-06-18 01:44:08.497145": preprocesing finished
"2018-06-18 01:44:08.497185": start training on days [0]
"2018-06-18 01:44:08.497200": using fit without validation
"2018-06-18 01:44:28.996665": built 1000 trees
"2018-06-18 01:44:28.996743": start predicting on day 6
"2018-06-18 01:44:46.233453": saveing results
"2018-06-18 01:44:46.241200": results saved
"2018-06-18 01:44:46.244973": predictions_filenames: ['res/train_0_test_6.npy']
"2018-06-18 01:44:46.245018": will evaluate days: [6]
"2018-06-18 01:44:46.245096": predict on file "res/train_0_test_6.npy"
"2018-06-18 01:44:48.212410": calculating metric
"2018-06-18 01:44:48.213440": preprocesing started
"2018-06-18 01:44:57.445193": train features shape: (22801, 80)
"2018-06-18 01:44:57.445277": preprocesing finished
"2018-06-18 01:44:57.445389": start training on days [0]
"2018-06-18 01:44:57.445404": using fit withou

"2018-06-18 01:53:20.830144": built 1000 trees
"2018-06-18 01:53:20.830231": start predicting on day 6
"2018-06-18 01:53:35.451702": saveing results
"2018-06-18 01:53:35.460338": results saved
"2018-06-18 01:53:35.464233": predictions_filenames: ['res/train_0_test_6.npy']
"2018-06-18 01:53:35.464281": will evaluate days: [6]
"2018-06-18 01:53:35.464399": predict on file "res/train_0_test_6.npy"
"2018-06-18 01:53:37.467086": calculating metric
"2018-06-18 01:53:37.468394": preprocesing started
"2018-06-18 01:53:47.354942": train features shape: (22801, 80)
"2018-06-18 01:53:47.355044": preprocesing finished
"2018-06-18 01:53:47.355062": start training on days [0]
"2018-06-18 01:53:47.355075": using fit without validation
"2018-06-18 01:54:06.616657": built 1000 trees
"2018-06-18 01:54:06.616743": start predicting on day 6
"2018-06-18 01:54:21.300092": saveing results
"2018-06-18 01:54:21.308183": results saved
"2018-06-18 01:54:21.312500": predictions_filenames: ['res/train_0_test_6.npy

"2018-06-18 02:02:00.020195": calculating metric
"2018-06-18 02:02:00.021693": preprocesing started
"2018-06-18 02:02:09.646602": train features shape: (22452, 80)
"2018-06-18 02:02:09.646703": preprocesing finished
"2018-06-18 02:02:09.646806": start training on days [1]
"2018-06-18 02:02:09.646821": using fit without validation
"2018-06-18 02:02:29.009149": built 1000 trees
"2018-06-18 02:02:29.009236": start predicting on day 6
"2018-06-18 02:02:43.613253": saveing results
"2018-06-18 02:02:43.621310": results saved
"2018-06-18 02:02:43.625251": predictions_filenames: ['res/train_1_test_6.npy']
"2018-06-18 02:02:43.625336": will evaluate days: [6]
"2018-06-18 02:02:43.625579": predict on file "res/train_1_test_6.npy"
"2018-06-18 02:02:45.677468": calculating metric
"2018-06-18 02:02:45.678519": preprocesing started
"2018-06-18 02:02:55.325227": train features shape: (22452, 80)
"2018-06-18 02:02:55.325328": preprocesing finished
"2018-06-18 02:02:55.325351": start training on days [

"2018-06-18 02:10:53.215857": built 1000 trees
"2018-06-18 02:10:53.215942": start predicting on day 6
"2018-06-18 02:11:08.121399": saveing results
"2018-06-18 02:11:08.130477": results saved
"2018-06-18 02:11:08.134505": predictions_filenames: ['res/train_2_test_6.npy']
"2018-06-18 02:11:08.134568": will evaluate days: [6]
"2018-06-18 02:11:08.134779": predict on file "res/train_2_test_6.npy"
"2018-06-18 02:11:10.154093": calculating metric
"2018-06-18 02:11:10.155231": preprocesing started
"2018-06-18 02:11:19.672771": train features shape: (22314, 80)
"2018-06-18 02:11:19.672857": preprocesing finished
"2018-06-18 02:11:19.672970": start training on days [2]
"2018-06-18 02:11:19.672986": using fit without validation
"2018-06-18 02:11:39.068925": built 1000 trees
"2018-06-18 02:11:39.069197": start predicting on day 6
"2018-06-18 02:11:53.959365": saveing results
"2018-06-18 02:11:53.967242": results saved
"2018-06-18 02:11:53.971205": predictions_filenames: ['res/train_2_test_6.npy

"2018-06-18 02:19:34.835984": calculating metric
"2018-06-18 02:19:34.837138": preprocesing started
"2018-06-18 02:19:44.413033": train features shape: (20203, 80)
"2018-06-18 02:19:44.413118": preprocesing finished
"2018-06-18 02:19:44.413231": start training on days [3]
"2018-06-18 02:19:44.413246": using fit without validation
"2018-06-18 02:20:02.960665": built 1000 trees
"2018-06-18 02:20:02.960771": start predicting on day 6
"2018-06-18 02:20:17.817548": saveing results
"2018-06-18 02:20:17.826004": results saved
"2018-06-18 02:20:17.829950": predictions_filenames: ['res/train_3_test_6.npy']
"2018-06-18 02:20:17.829997": will evaluate days: [6]
"2018-06-18 02:20:17.830112": predict on file "res/train_3_test_6.npy"
"2018-06-18 02:20:19.830841": calculating metric
"2018-06-18 02:20:19.832048": preprocesing started
"2018-06-18 02:20:29.245408": train features shape: (20203, 80)
"2018-06-18 02:20:29.245511": preprocesing finished
"2018-06-18 02:20:29.245612": start training on days [

...

"2018-06-18 05:30:11.710185": calculating metric
"2018-06-18 05:30:11.711627": preprocesing started
"2018-06-18 05:30:21.133541": train features shape: (102173, 80)
"2018-06-18 05:30:21.133632": preprocesing finished
"2018-06-18 05:30:21.133654": start training on days [0, 1, 2, 3, 4]
"2018-06-18 05:30:21.133682": using fit without validation
"2018-06-18 05:30:55.607260": built 1000 trees
"2018-06-18 05:30:55.607374": start predicting on day 6
"2018-06-18 05:31:11.201342": saveing results
"2018-06-18 05:31:11.209797": results saved
"2018-06-18 05:31:11.213819": predictions_filenames: ['res/train_0_1_2_3_4_test_6.npy']
"2018-06-18 05:31:11.213864": will evaluate days: [6]
"2018-06-18 05:31:11.214022": predict on file "res/train_0_1_2_3_4_test_6.npy"
"2018-06-18 05:31:13.247505": calculating metric
"2018-06-18 05:31:13.248585": preprocesing started
"2018-06-18 05:31:22.684450": train features shape: (102173, 80)
"2018-06-18 05:31:22.684551": preprocesing finished
"2018-06-18 05:31:22.684

"2018-06-18 05:41:25.631347": calculating metric
"2018-06-18 05:41:25.632492": preprocesing started
"2018-06-18 05:41:35.068618": train features shape: (102173, 80)
"2018-06-18 05:41:35.068706": preprocesing finished
"2018-06-18 05:41:35.068728": start training on days [0, 1, 2, 3, 4]
"2018-06-18 05:41:35.068758": using fit without validation
"2018-06-18 05:42:09.572764": built 1000 trees
"2018-06-18 05:42:09.572855": start predicting on day 6
"2018-06-18 05:42:24.660401": saveing results
"2018-06-18 05:42:24.668153": results saved
"2018-06-18 05:42:24.672164": predictions_filenames: ['res/train_0_1_2_3_4_test_6.npy']
"2018-06-18 05:42:24.672216": will evaluate days: [6]
"2018-06-18 05:42:24.672375": predict on file "res/train_0_1_2_3_4_test_6.npy"
"2018-06-18 05:42:26.696477": calculating metric
"2018-06-18 05:42:26.697553": preprocesing started
"2018-06-18 05:42:36.180721": train features shape: (102173, 80)
"2018-06-18 05:42:36.180841": preprocesing finished
"2018-06-18 05:42:36.180

"2018-06-18 05:52:20.532355": calculating metric
"2018-06-18 05:52:20.533823": preprocesing started
"2018-06-18 05:52:29.990314": train features shape: (95119, 80)
"2018-06-18 05:52:29.990401": preprocesing finished
"2018-06-18 05:52:29.990613": start training on days [1, 2, 3, 4, 5]
"2018-06-18 05:52:29.990629": using fit without validation
"2018-06-18 05:53:02.073355": built 1000 trees
"2018-06-18 05:53:02.073464": start predicting on day 6
"2018-06-18 05:53:17.147371": saveing results
"2018-06-18 05:53:17.155550": results saved
"2018-06-18 05:53:17.159763": predictions_filenames: ['res/train_1_2_3_4_5_test_6.npy']
"2018-06-18 05:53:17.159928": will evaluate days: [6]
"2018-06-18 05:53:17.160085": predict on file "res/train_1_2_3_4_5_test_6.npy"
"2018-06-18 05:53:19.204872": calculating metric
"2018-06-18 05:53:19.206266": preprocesing started
"2018-06-18 05:53:28.718468": train features shape: (95119, 80)
"2018-06-18 05:53:28.718604": preprocesing finished
"2018-06-18 05:53:28.71880

"2018-06-18 06:03:37.924792": calculating metric
"2018-06-18 06:03:37.925942": preprocesing started
"2018-06-18 06:03:47.461653": train features shape: (117920, 80)
"2018-06-18 06:03:47.461740": preprocesing finished
"2018-06-18 06:03:47.461851": start training on days [0, 1, 2, 3, 4, 5]
"2018-06-18 06:03:47.461866": using fit without validation
"2018-06-18 06:04:27.146861": built 1000 trees
"2018-06-18 06:04:27.146953": start predicting on day 6
"2018-06-18 06:04:42.401150": saveing results
"2018-06-18 06:04:42.409372": results saved
"2018-06-18 06:04:42.413788": predictions_filenames: ['res/train_0_1_2_3_4_5_test_6.npy']
"2018-06-18 06:04:42.413839": will evaluate days: [6]
"2018-06-18 06:04:42.413989": predict on file "res/train_0_1_2_3_4_5_test_6.npy"
"2018-06-18 06:04:44.464020": calculating metric
"2018-06-18 06:04:44.465082": preprocesing started
"2018-06-18 06:04:53.897958": train features shape: (117920, 80)
"2018-06-18 06:04:53.898046": preprocesing finished
"2018-06-18 06:04

"2018-06-18 06:15:50.943065": calculating metric
CPU times: user 19h 21min 29s, sys: 1h 15min 12s, total: 20h 36min 42s
Wall time: 4h 31min 52s


In [5]:
%%time
no_val_metrics = {}
for train_variant in train_variants:
    metric = [get_baseline_metric(train_variant) for i in range(15)]
    no_val_metrics[str(train_variant)] = metric

"2018-06-18 06:15:50.956515": preprocesing started
"2018-06-18 06:16:00.531298": train features shape: (22801, 80)
"2018-06-18 06:16:00.531435": preprocesing finished
"2018-06-18 06:16:00.531775": start training on days [0]
"2018-06-18 06:16:00.531843": using fit without validation
"2018-06-18 06:16:19.797173": built 1000 trees
"2018-06-18 06:16:19.797280": start predicting on day 6
"2018-06-18 06:16:34.637067": saveing results
"2018-06-18 06:16:34.644891": results saved
"2018-06-18 06:16:34.648477": predictions_filenames: ['res/train_0_test_6.npy']
"2018-06-18 06:16:34.648596": will evaluate days: [6]
"2018-06-18 06:16:34.648801": predict on file "res/train_0_test_6.npy"
"2018-06-18 06:16:36.658943": calculating metric
"2018-06-18 06:16:36.659911": preprocesing started
"2018-06-18 06:16:46.112726": train features shape: (22801, 80)
"2018-06-18 06:16:46.112827": preprocesing finished
"2018-06-18 06:16:46.113173": start training on days [0]
"2018-06-18 06:16:46.113320": using fit withou

"2018-06-18 06:24:44.164596": built 1000 trees
"2018-06-18 06:24:44.164684": start predicting on day 6
"2018-06-18 06:24:59.489458": saveing results
"2018-06-18 06:24:59.497654": results saved
"2018-06-18 06:24:59.501289": predictions_filenames: ['res/train_0_test_6.npy']
"2018-06-18 06:24:59.501478": will evaluate days: [6]
"2018-06-18 06:24:59.501610": predict on file "res/train_0_test_6.npy"
"2018-06-18 06:25:01.531445": calculating metric
"2018-06-18 06:25:01.532563": preprocesing started
"2018-06-18 06:25:10.962777": train features shape: (22801, 80)
"2018-06-18 06:25:10.962890": preprocesing finished
"2018-06-18 06:25:10.963013": start training on days [0]
"2018-06-18 06:25:10.963043": using fit without validation
"2018-06-18 06:25:30.141728": built 1000 trees
"2018-06-18 06:25:30.141812": start predicting on day 6
"2018-06-18 06:25:45.349538": saveing results
"2018-06-18 06:25:45.358022": results saved
"2018-06-18 06:25:45.361948": predictions_filenames: ['res/train_0_test_6.npy

"2018-06-18 06:33:26.526839": calculating metric
"2018-06-18 06:33:26.527974": preprocesing started
"2018-06-18 06:33:35.916448": train features shape: (22452, 80)
"2018-06-18 06:33:35.916532": preprocesing finished
"2018-06-18 06:33:35.916566": start training on days [1]
"2018-06-18 06:33:35.916580": using fit without validation
"2018-06-18 06:33:55.145219": built 1000 trees
"2018-06-18 06:33:55.145307": start predicting on day 6
"2018-06-18 06:34:10.389759": saveing results
"2018-06-18 06:34:10.397777": results saved
"2018-06-18 06:34:10.401542": predictions_filenames: ['res/train_1_test_6.npy']
"2018-06-18 06:34:10.401589": will evaluate days: [6]
"2018-06-18 06:34:10.401743": predict on file "res/train_1_test_6.npy"
"2018-06-18 06:34:12.422289": calculating metric
"2018-06-18 06:34:12.423555": preprocesing started
"2018-06-18 06:34:21.900587": train features shape: (22452, 80)
"2018-06-18 06:34:21.900690": preprocesing finished
"2018-06-18 06:34:21.900875": start training on days [

"2018-06-18 06:42:19.351916": built 1000 trees
"2018-06-18 06:42:19.352011": start predicting on day 6
"2018-06-18 06:42:34.108824": saveing results
"2018-06-18 06:42:34.116752": results saved
"2018-06-18 06:42:34.120675": predictions_filenames: ['res/train_2_test_6.npy']
"2018-06-18 06:42:34.120726": will evaluate days: [6]
"2018-06-18 06:42:34.120907": predict on file "res/train_2_test_6.npy"
"2018-06-18 06:42:36.151393": calculating metric
"2018-06-18 06:42:36.152421": preprocesing started
"2018-06-18 06:42:45.677725": train features shape: (22314, 80)
"2018-06-18 06:42:45.677824": preprocesing finished
"2018-06-18 06:42:45.678056": start training on days [2]
"2018-06-18 06:42:45.678077": using fit without validation
"2018-06-18 06:43:04.897214": built 1000 trees
"2018-06-18 06:43:04.897301": start predicting on day 6
"2018-06-18 06:43:20.255932": saveing results
"2018-06-18 06:43:20.264083": results saved
"2018-06-18 06:43:20.267837": predictions_filenames: ['res/train_2_test_6.npy

"2018-06-18 06:50:58.171923": calculating metric
"2018-06-18 06:50:58.173172": preprocesing started
"2018-06-18 06:51:07.457308": train features shape: (20203, 80)
"2018-06-18 06:51:07.457406": preprocesing finished
"2018-06-18 06:51:07.457432": start training on days [3]
"2018-06-18 06:51:07.457458": using fit without validation
"2018-06-18 06:51:25.803527": built 1000 trees
"2018-06-18 06:51:25.803596": start predicting on day 6
"2018-06-18 06:51:41.381078": saveing results
"2018-06-18 06:51:41.389003": results saved
"2018-06-18 06:51:41.392853": predictions_filenames: ['res/train_3_test_6.npy']
"2018-06-18 06:51:41.392912": will evaluate days: [6]
"2018-06-18 06:51:41.393069": predict on file "res/train_3_test_6.npy"
"2018-06-18 06:51:43.383195": calculating metric
"2018-06-18 06:51:43.384365": preprocesing started
"2018-06-18 06:51:52.771180": train features shape: (20203, 80)
"2018-06-18 06:51:52.771262": preprocesing finished
"2018-06-18 06:51:52.771467": start training on days [

...

"2018-06-18 10:02:10.749554": saveing results
"2018-06-18 10:02:10.757172": results saved
"2018-06-18 10:02:10.760936": predictions_filenames: ['res/train_0_1_2_3_4_test_6.npy']
"2018-06-18 10:02:10.761006": will evaluate days: [6]
"2018-06-18 10:02:10.761164": predict on file "res/train_0_1_2_3_4_test_6.npy"
"2018-06-18 10:02:12.786326": calculating metric
"2018-06-18 10:02:12.787330": preprocesing started
"2018-06-18 10:02:22.170981": train features shape: (102173, 80)
"2018-06-18 10:02:22.171079": preprocesing finished
"2018-06-18 10:02:22.171200": start training on days [0, 1, 2, 3, 4]
"2018-06-18 10:02:22.171313": using fit without validation
"2018-06-18 10:02:56.623247": built 1000 trees
"2018-06-18 10:02:56.623338": start predicting on day 6
"2018-06-18 10:03:11.599750": saveing results
"2018-06-18 10:03:11.607888": results saved
"2018-06-18 10:03:11.612116": predictions_filenames: ['res/train_0_1_2_3_4_test_6.npy']
"2018-06-18 10:03:11.612165": will evaluate days: [6]
"2018-06-

"2018-06-18 10:13:17.420296": built 1000 trees
"2018-06-18 10:13:17.420384": start predicting on day 6
"2018-06-18 10:13:32.147236": saveing results
"2018-06-18 10:13:32.155152": results saved
"2018-06-18 10:13:32.159395": predictions_filenames: ['res/train_0_1_2_3_4_test_6.npy']
"2018-06-18 10:13:32.159534": will evaluate days: [6]
"2018-06-18 10:13:32.159724": predict on file "res/train_0_1_2_3_4_test_6.npy"
"2018-06-18 10:13:34.054674": calculating metric
"2018-06-18 10:13:34.055675": preprocesing started
"2018-06-18 10:13:43.112262": train features shape: (102173, 80)
"2018-06-18 10:13:43.112335": preprocesing finished
"2018-06-18 10:13:43.112353": start training on days [0, 1, 2, 3, 4]
"2018-06-18 10:13:43.112446": using fit without validation
"2018-06-18 10:14:16.688423": built 1000 trees
"2018-06-18 10:14:16.688517": start predicting on day 6
"2018-06-18 10:14:33.134058": saveing results
"2018-06-18 10:14:33.142520": results saved
"2018-06-18 10:14:33.146185": predictions_filena

"2018-06-18 10:24:08.256397": built 1000 trees
"2018-06-18 10:24:08.256488": start predicting on day 6
"2018-06-18 10:24:22.411787": saveing results
"2018-06-18 10:24:22.419588": results saved
"2018-06-18 10:24:22.423754": predictions_filenames: ['res/train_1_2_3_4_5_test_6.npy']
"2018-06-18 10:24:22.423806": will evaluate days: [6]
"2018-06-18 10:24:22.423942": predict on file "res/train_1_2_3_4_5_test_6.npy"
"2018-06-18 10:24:24.460475": calculating metric
"2018-06-18 10:24:24.461672": preprocesing started
"2018-06-18 10:24:33.568151": train features shape: (95119, 80)
"2018-06-18 10:24:33.568235": preprocesing finished
"2018-06-18 10:24:33.568488": start training on days [1, 2, 3, 4, 5]
"2018-06-18 10:24:33.568546": using fit without validation
"2018-06-18 10:25:05.051815": built 1000 trees
"2018-06-18 10:25:05.051964": start predicting on day 6
"2018-06-18 10:25:19.121083": saveing results
"2018-06-18 10:25:19.128759": results saved
"2018-06-18 10:25:19.132399": predictions_filenam

"2018-06-18 10:35:15.601406": built 1000 trees
"2018-06-18 10:35:15.601684": start predicting on day 6
"2018-06-18 10:35:30.961557": saveing results
"2018-06-18 10:35:30.969846": results saved
"2018-06-18 10:35:30.974742": predictions_filenames: ['res/train_0_1_2_3_4_5_test_6.npy']
"2018-06-18 10:35:30.974796": will evaluate days: [6]
"2018-06-18 10:35:30.974911": predict on file "res/train_0_1_2_3_4_5_test_6.npy"
"2018-06-18 10:35:33.063378": calculating metric
"2018-06-18 10:35:33.064505": preprocesing started
"2018-06-18 10:35:42.428705": train features shape: (117920, 80)
"2018-06-18 10:35:42.428791": preprocesing finished
"2018-06-18 10:35:42.428957": start training on days [0, 1, 2, 3, 4, 5]
"2018-06-18 10:35:42.428972": using fit without validation
"2018-06-18 10:36:22.204554": built 1000 trees
"2018-06-18 10:36:22.204647": start predicting on day 6
"2018-06-18 10:36:37.181480": saveing results
"2018-06-18 10:36:37.189645": results saved
"2018-06-18 10:36:37.195365": predictions

"2018-06-18 10:46:47.465385": preprocesing finished
"2018-06-18 10:46:47.465505": start training on days [0, 1, 2, 3, 4, 5]
"2018-06-18 10:46:47.465582": using fit without validation
"2018-06-18 10:47:27.422567": built 1000 trees
"2018-06-18 10:47:27.422659": start predicting on day 6
"2018-06-18 10:47:42.700777": saveing results
"2018-06-18 10:47:42.708676": results saved
"2018-06-18 10:47:42.715488": predictions_filenames: ['res/train_0_1_2_3_4_5_test_6.npy']
"2018-06-18 10:47:42.715570": will evaluate days: [6]
"2018-06-18 10:47:42.715738": predict on file "res/train_0_1_2_3_4_5_test_6.npy"
"2018-06-18 10:47:44.771576": calculating metric
CPU times: user 19h 22min 13s, sys: 1h 14min 24s, total: 20h 36min 37s
Wall time: 4h 31min 53s
